In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ipywidgets import *

In [2]:
movies=pd.read_csv("C:\\Users\\anagh\\Downloads\\movies (1).csv")

In [3]:
ratings=pd.read_csv("C:\\Users\\anagh\\Downloads\\ratings.csv")

In [4]:
movies.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

In [5]:
ratings.head()

userId  movieId  rating   timestamp
0       1       16     4.0  1217897793
1       1       24     1.5  1217895807
2       1       32     4.0  1217896246
3       1       47     4.0  1217896556
4       1       50     4.0  1217896523

In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10329 entries, 0 to 10328
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  10329 non-null  int64 
 1   title    10329 non-null  object
 2   genres   10329 non-null  object
dtypes: int64(1), object(2)
memory usage: 242.2+ KB


In [7]:
movies.shape

(10329, 3)

In [8]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105339 entries, 0 to 105338
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     105339 non-null  int64  
 1   movieId    105339 non-null  int64  
 2   rating     105339 non-null  float64
 3   timestamp  105339 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.2 MB


In [9]:
ratings.shape

(105339, 4)

In [10]:
ratings.describe()

userId        movieId         rating     timestamp
count  105339.000000  105339.000000  105339.000000  1.053390e+05
mean      364.924539   13381.312477       3.516850  1.130424e+09
std       197.486905   26170.456869       1.044872  1.802660e+08
min         1.000000       1.000000       0.500000  8.285650e+08
25%       192.000000    1073.000000       3.000000  9.711008e+08
50%       383.000000    2497.000000       3.500000  1.115154e+09
75%       557.000000    5991.000000       4.000000  1.275496e+09
max       668.000000  149532.000000       5.000000  1.452405e+09

In [11]:
movies['genres']=movies['genres'].str.split("|")

In [12]:
movies.explode('genres')

movieId                                title              genres
0            1                     Toy Story (1995)           Adventure
0            1                     Toy Story (1995)           Animation
0            1                     Toy Story (1995)            Children
0            1                     Toy Story (1995)              Comedy
0            1                     Toy Story (1995)             Fantasy
...        ...                                  ...                 ...
10324   146684        Cosmic Scrat-tastrophe (2015)              Comedy
10325   146878           Le Grand Restaurant (1966)              Comedy
10326   148238       A Very Murray Christmas (2015)              Comedy
10327   148626                 The Big Short (2015)               Drama
10328   149532  Marco Polo: One Hundred Eyes (2015)  (no genres listed)

[23114 rows x 3 columns]

In [13]:
movies2=movies.explode('genres')
movies2.head()

movieId             title     genres
0        1  Toy Story (1995)  Adventure
0        1  Toy Story (1995)  Animation
0        1  Toy Story (1995)   Children
0        1  Toy Story (1995)     Comedy
0        1  Toy Story (1995)    Fantasy

In [14]:
movies2['genres'].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'War', 'Musical', 'Documentary',
       'Western', 'Film-Noir', '(no genres listed)'], dtype=object)

In [15]:
movies2['genres'].nunique()

20

In [16]:
movies2=movies2[movies2['genres']!='(no genres listed)']

In [17]:
movies2['genres'].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'War', 'Musical', 'Documentary',
       'Western', 'Film-Noir'], dtype=object)

In [18]:
movies2['genres'].nunique()

19

In [19]:
merged_data=pd.merge(ratings,movies2,on=['movieId'],how='inner')
merged_data

userId  movieId  rating   timestamp  \
0            1       16     4.0  1217897793   
1            1       16     4.0  1217897793   
2            1       24     1.5  1217895807   
3            1       24     1.5  1217895807   
4            1       32     4.0  1217896246   
...        ...      ...     ...         ...   
281892     668   143385     4.0  1446388585   
281893     668   143385     4.0  1446388585   
281894     668   144976     2.5  1448656898   
281895     668   144976     2.5  1448656898   
281896     668   148626     4.5  1451148148   

                                            title    genres  
0                                   Casino (1995)     Crime  
1                                   Casino (1995)     Drama  
2                                   Powder (1995)     Drama  
3                                   Powder (1995)    Sci-Fi  
4       Twelve Monkeys (a.k.a. 12 Monkeys) (1995)   Mystery  
...                                           ...       ...  
281892                     Bridge of Spies (2015)     Drama  
281893                     Bridge of Spies (2015)  Thriller  
281894                       Bone Tomahawk (2015)    Horror  
281895                       Bone Tomahawk (2015)   Western  
281896                       The Big Short (2015)     Drama  

[281897 rows x 6 columns]

In [20]:
merged_data.groupby('genres').agg({"title":"nunique",'rating':['mean','size']}).rename(columns={"title":"unique_movie_count","rating":"mean_ratings"})

unique_movie_count mean_ratings       
                       nunique         mean   size
genres                                            
Action                    1736     3.451450  31205
Adventure                 1164     3.518027  23076
Animation                  400     3.635350   5966
Children                   540     3.439429   8098
Comedy                    3513     3.420996  38055
Crime                     1440     3.642392  18291
Documentary                415     3.643035   1206
Drama                     5217     3.650266  46960
Fantasy                    670     3.500459  10889
Film-Noir                  195     3.913636   1210
Horror                    1001     3.281097   7983
IMAX                       152     3.641821   3032
Musical                    409     3.571962   4287
Mystery                    675     3.652043   8320
Romance                   1788     3.544255  19094
Sci-Fi                     858     3.454481  16795
Thriller                  2187     3.495561  29288
War                        503     3.783202   5828
Western                    235     3.565687   2314

In [21]:
popularity=merged_data.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()
popularity.columns=["Genres","Title","Average_Ratings","Number_of_Ratings"]
popularity

Genres                                       Title  Average_Ratings  \
0       Action                                  '71 (2014)         3.500000   
1       Action     'Hellboy': The Seeds of Creation (2004)         3.000000   
2       Action                       10 to Midnight (1983)         2.500000   
3       Action                            12 Rounds (2009)         2.875000   
4       Action  13 Assassins (Jûsan-nin no shikaku) (2010)         3.500000   
...        ...                                         ...              ...   
23093  Western                           Wyatt Earp (1994)         3.200000   
23094  Western                           Young Guns (1988)         3.375000   
23095  Western                        Young Guns II (1990)         3.083333   
23096  Western                           Young Ones (2014)         2.000000   
23097  Western                       ¡Three Amigos! (1986)         3.012500   

       Number_of_Ratings  
0                      1  
1                      1  
2                      1  
3                      4  
4                      5  
...                  ...  
23093                 30  
23094                 36  
23095                 12  
23096                  1  
23097                 40  

[23098 rows x 4 columns]

In [22]:
popularity[(popularity['Genres']=='Action')&(popularity['Number_of_Ratings']>=50)].sort_values(by='Average_Ratings',ascending=False).head(7)

Genres                                              Title  \
1179  Action           Princess Mononoke (Mononoke-hime) (1997)   
1076  Action                          North by Northwest (1959)   
975   Action                                 Matrix, The (1999)   
1433  Action  Star Wars: Episode V - The Empire Strikes Back...   
1331  Action        Seven Samurai (Shichinin no samurai) (1954)   
1199  Action  Raiders of the Lost Ark (Indiana Jones and the...   
747   Action                                   Inception (2010)   

      Average_Ratings  Number_of_Ratings  
1179         4.384615                 52  
1076         4.273973                 73  
975          4.264368                261  
1433         4.228070                228  
1331         4.217742                 62  
1199         4.212054                224  
747          4.189320                103

In [23]:
def TopNPopularMovies(genre,threshold,topN):
    popularity=merged_data.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()
    popularity.columns=["Genres","Title","Average_Ratings","Number_of_Ratings"]
    topNrecommendations=popularity[(popularity['Genres']==genre)&(popularity['Number_of_Ratings']>=threshold)].sort_values(by='Average_Ratings',ascending=False).head(topN)
    topNrecommendations['Sno.']=range(1,len(topNrecommendations)+1)
    topNrecommendations.index = range(0, len(topNrecommendations))
    topNrecommendations.columns=['Genres','Movie Title','Average Movie Rating','Number of Reviews','Sno.']
    return topNrecommendations[['Sno.','Movie Title','Average Movie Rating','Number of Reviews']]

In [24]:
genre='Adventure'
threshold=50
topN=8
TopNPopularMovies(genre='Adventure',threshold=50,topN=8)

Sno.                                        Movie Title  \
0     1           Princess Mononoke (Mononoke-hime) (1997)   
1     2             Monty Python and the Holy Grail (1975)   
2     3                          North by Northwest (1959)   
3     4  Spirited Away (Sen to Chihiro no kamikakushi) ...   
4     5  Star Wars: Episode V - The Empire Strikes Back...   
5     6        Seven Samurai (Shichinin no samurai) (1954)   
6     7  Raiders of the Lost Ark (Indiana Jones and the...   
7     8          Star Wars: Episode IV - A New Hope (1977)   

   Average Movie Rating  Number of Reviews  
0              4.384615                 52  
1              4.301948                154  
2              4.273973                 73  
3              4.236111                 72  
4              4.228070                228  
5              4.217742                 62  
6              4.212054                224  
7              4.188645                273

In [25]:
movies2.head()

movieId             title     genres
0        1  Toy Story (1995)  Adventure
0        1  Toy Story (1995)  Animation
0        1  Toy Story (1995)   Children
0        1  Toy Story (1995)     Comedy
0        1  Toy Story (1995)    Fantasy

In [26]:
movies3=movies2.groupby('title').agg({"genres":lambda x:" ".join(list(x))}).reset_index()

In [27]:
movies3.head()

title  \
0                               '71 (2014)   
1  'Hellboy': The Seeds of Creation (2004)   
2                   'Round Midnight (1986)   
3                'Til There Was You (1997)   
4                       'burbs, The (1989)   

                                        genres  
0                    Action Drama Thriller War  
1  Action Adventure Comedy Documentary Fantasy  
2                                Drama Musical  
3                                Drama Romance  
4                                       Comedy

In [28]:
tf=TfidfVectorizer(analyzer='word',ngram_range=(1,3),stop_words='english')
tf

TfidfVectorizer(ngram_range=(1, 3), stop_words='english')

In [29]:
tf_matrix=tf.fit_transform(movies3['genres'])

In [30]:
cosine_sim=cosine_similarity(tf_matrix,tf_matrix)

In [31]:
cosine_sim

array([[1.        , 0.02677945, 0.02931913, ..., 0.10229517, 0.        ,
        0.        ],
       [0.02677945, 1.        , 0.        , ..., 0.03626651, 0.02411583,
        0.02863994],
       [0.02931913, 0.        , 1.        , ..., 0.        , 0.        ,
        0.35526663],
       ...,
       [0.10229517, 0.03626651, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.02411583, 0.        , ..., 0.        , 1.        ,
        0.07090711],
       [0.        , 0.02863994, 0.35526663, ..., 0.        , 0.07090711,
        1.        ]])

In [32]:
indices=pd.Series(movies3.index,index=movies3['title'])
indices

title
'71 (2014)                                           0
'Hellboy': The Seeds of Creation (2004)              1
'Round Midnight (1986)                               2
'Til There Was You (1997)                            3
'burbs, The (1989)                                   4
                                                 ...  
loudQUIETloud: A Film About the Pixies (2006)    10315
xXx (2002)                                       10316
xXx: State of the Union (2005)                   10317
¡Three Amigos! (1986)                            10318
À nous la liberté (Freedom for Us) (1931)        10319
Length: 10320, dtype: int64

In [33]:
index=indices["'71 (2014)"]

In [34]:
scores=list(enumerate(cosine_sim[index]))

In [35]:
matched=sorted(scores,key=lambda x:x[1],reverse=True)[:5]

In [36]:
matched=[i[0] for i in matched]
matched

[0, 596, 858, 3839, 4407]

In [37]:
movies3.iloc[matched]

title  \
0                                            '71 (2014)   
596         Army of Shadows (L'armée des ombres) (1969)   
858   Battle Royale 2: Requiem (Batoru rowaiaru II: ...   
3839                                  Green Zone (2010)   
4407                            Hurt Locker, The (2008)   

                         genres  
0     Action Drama Thriller War  
596   Action Drama Thriller War  
858   Action Drama Thriller War  
3839  Action Drama Thriller War  
4407  Action Drama Thriller War

In [38]:
def recommendation_genre(movie_df,similarity_matrix,movie_title,topN):
    indices=pd.Series(movies3.index,index=movies3['title'])
    index=indices[movie_title]
    cosine_scores=list(enumerate(similarity_matrix[index]))
    cosine_scores=sorted(cosine_scores,key=lambda x:x[1],reverse=True)[1:topN+2]
    matched=[i[0] for i in cosine_scores]
    matching_df=movie_df.iloc[matched]
    matching_df=matching_df[matching_df['title']!=movie_title]
    matching_df.rename(columns={'title':'Movie Title'},inplace=True)
    matching_df['Sno.']=range(1,len(matching_df)+1)
    matching_df.index=range(0,len(matching_df))
    return matching_df[['Sno.','Movie Title']].head(topN)
    

In [39]:
recommendation_genre(movie_df=movies3,similarity_matrix=cosine_sim,movie_title='Toy Story (1995)',topN=10)

Sno.                                        Movie Title
0     1                                        Antz (1998)
1     2  Asterix and the Vikings (Astérix et les Viking...
2     3                              Boxtrolls, The (2014)
3     4  DuckTales: The Movie - Treasure of the Lost La...
4     5                   Emperor's New Groove, The (2000)
5     6                              Monsters, Inc. (2001)
6     7                             Shrek the Third (2007)
7     8                     Tale of Despereaux, The (2008)
8     9                                 Toy Story 2 (1999)
9    10                                       Turbo (2013)

In [40]:
genres=Dropdown(options=list(set(movies2['genres'])),description="Genres",style={"description_width":'initial'})
num_reviews=IntText(description="Minimum Reviews",style={"description_width":'initial'})
num_recommendations_1=IntText(description="Number of Recommendations",style={"description_width":'initial'})
b1=Button(description="RECOMMEND ME",style={"description_width":'initial'})
h1=HBox([num_reviews,num_recommendations_1])
popularity_tab=VBox([genres,h1,b1])


title=Textarea(description="Movie Title",style={"description_width":'initial'})
num_recommendations_2=IntText(description="Number of Recommendations",style={"description_width":'initial'})
h2=HBox([title,num_recommendations_2])
b2=Button(description="RECOMMEND ME",style={"description_width":'initial'})
  
content_tab=VBox([h2,b2])

tabs=[popularity_tab,content_tab]
wid=widgets.Tab(tabs)

names=['Popularity Based Recommendations','Content Based Recommendations']
[wid.set_title(i,title) for i,title in enumerate(names)]
display(wid)

In [47]:
def b1_clicked(b):
    global output
    output=TopNPopularMovies(genre=genres.value,threshold=num_reviews.value,topN=num_recommendations_1.value)
b1.on_click(b1_clicked)

def b2_clicked(b):
    global output
    result=recommendation_genre(movie_df=movies3,similarity_matrix=cosine_sim,movie_title=title.value,topN=num_recommendations_2.value)
    output=result
b2.on_click(b2_clicked)

In [48]:
display(wid)

In [52]:
output

Sno.                                        Movie Title  \
0     1  Nausicaä of the Valley of the Wind (Kaze no ta...   
1     2                                  About Time (2013)   
2     3           Princess Mononoke (Mononoke-hime) (1997)   
3     4       My Neighbor Totoro (Tonari no Totoro) (1988)   
4     5             Monty Python and the Holy Grail (1975)   

   Average Movie Rating  Number of Reviews  
0              4.477273                 22  
1              4.416667                  6  
2              4.384615                 52  
3              4.321429                 28  
4              4.301948                154